<a href="https://colab.research.google.com/github/JungYoonJae04/IWantToGoUNIST/blob/main/RSA%EC%95%94%ED%98%B8%ED%99%94_2021_05_13~.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**등차수열을 이용한 RSA 암호화 개선**

보통의 RSA암호화는 큰 숫자를 소인수분해하기 힘든 특징을 이용하여 만든 암호체계이다. 그러나 결국 소인수분해가 성립하면 RSA 암호화는 무효화되고 보안이 개질 가능성이 있다. 대표적으로 RSA 암호화는 양자 컴퓨터가 등장하면 깨진다는 보안 위협에 직면해 있다. 1994년 피터 쇼어가 제안한 양자 알고리즘(쇼어 알고리즘)은 소인수 분해를 다항 시간안에 할 수 있는 양자 알고리즘이다. 따라서 RSA 암호화에 사용된 비트 수를 계산할 수 있을 정도의 충분한 큐비트 수를 가진 양자 컴퓨터가 개발되면 RSA 암호화 체계는 붕괴될 수 있다.

그래서 RSA암호화에서 생성된 숫자를 등차수열을 사용한 이동암호로 한번 더 엮어 애초에 소인수 분해할 수가 만들어지지 않도록 직접 코드를 구정해보았다.

Server와 Client의 역할은 다음과 같다.
먼저 Server는 등차수열에 사용될 초항과 등차를 랜덤으로 생성한 후, 기존의 RSA암호화를 통해 Client에게 개인키와 암호화된 초항과 등차를 전송한다. 그러면 앞으로 Server은 Client에게 암호문을 보낼 때 아까 전에 생성한 랜덤한 등차수열의 a(n)에 암호문을 대입하여 전송할 것이다. 그러면 Client는 암호문을 받은 후 등차수열을 역연산하여 원래의 암호문을 기존 RSA를 통해 해독하면 된다.

그러면 도청자는 어떤 소인수분해를 할 수 있더라도 이동암호가 이미 RSA를 통해 생성된 암호문을 뒤섞어놨기에 암호를 해독할 수 없다.

물론 여기에도 문제점은 존재한다. 처음에 초항과 등차를 전송할때에는 기존의 RSA암호화만을 적용하여 전송하기 때문에 이동암호가 소용없게 될 수 있지만, 초항과 등차만 조심히 잘 전달했다면, 초항과 등차를 주기적으로 바꿔주는 식으로 도청을 지속적으로 막을 수 있다고 생각한다.


1. Server에서의 이동암호 등차수열 생성

In [ ]:
#Server: 초기 수열 생성 (등차수열) a*(n-1)d
import random                             
alist=[]                                 
for i in range(2): 
  y = random.randint(1,1000)  
  alist.append(y)   
a1 = alist[0]
d2 = alist[1]
print("초항: %d, 등차: %d"% (int(a1), int(d2)))

초항: 29, 등차: 681


2. Server: 수열을 암호화하여 Client로 송신

In [ ]:
#Server: 수열을 암호화하여 Client로 송신
result = [a1, d2]
length = len(result) - 1
from random import *
print("공개키 및 개인키 생성")
p = int(input("첫번째 소수를 입력 : "))
q = int(input("두번째 소수를 입력 : "))
n = p*q
a = 2
pin =  (p-1)*(q-1)
data = []
so = []
while a < pin:
  for i in range(1, pin + 1):
    if (pin % i == 0) & (a % i == 0):
      data.append(i)
      gcd = i

  if gcd == 1:
    so.append(a)
    a = a +1
  else:
    a = a +13
k = len(so)
i = randrange(k) 
e = so[i]
d = 1
while (e*d)%pin != 1:
  d = d + 1
if e == d:
  d = d + 1
  while (e*d)%pin != 1:
    d = d + 1
print("공개키: (%d, %d)"% (int(n), int(e)))
print("개인키: (%d, %d)"% (int(n), int(d)))
print("공개키 (n, e)")
print("개인키 (n, d)")
print("n의 값 ", n)
print("e의 값 ", e)
print("d의 값 ", d)
codef = []
num = 0
while num <= length :
  M = result[num]
  C = (M**e) % n
  codef.append(C)
  num = num + 1
print("RSA 암호화 결과")
print(codef)

공개키 및 개인키 생성
첫번째 소수를 입력 : 13
두번째 소수를 입력 : 131
공개키: (1703, 161)
개인키: (1703, 281)
공개키 (n, e)
개인키 (n, d)
n의 값  1703
e의 값  161
d의 값  281
RSA 암호화 결과
[1283, 1032]


3. Client: 암호화된 문자를 해독하여 초항, 등ck값을 얻는다

In [ ]:
#Client: 암호화된 문자를 해독하여 2차 보안 수열값을 얻는다
print("개인키 입력 과정")
n = int(input("n값을 입력 : "))
d = int(input("d값을 입력 : "))
decode = []
kal = 0
print(codef)
while kal <= length:
  C = codef[kal]
  M = (C**d) % n
  decode.append(int(M))
  kal = kal + 1
print(decode)
print("an = a + (n-1)xd 에서")
print("초항: %d, 등차: %d"% (int(decode[0]), int(decode[1])))

개인키 입력 과정
n값을 입력 : 1703
d값을 입력 : 281
[1283, 1032]
[29, 681]
an = a + (n-1)xd 에서
초항: 29, 등차: 681


4. Server: 송신할 정보를 RSA로 암호화 한 후 등차수열에 대입하여 결과값을 전송

In [ ]:
#Server: 글을 암호화하여 Client에게 송신
print("암호화할 문자 입력[가급적 영어로]:")
s = input("Enter : ")
result = []
for i in s:
  result.append(ord(i))
length = len(result) - 1
print("아스키코드로 변환")
print(result)
#공개키, 개인키 생성
from random import *
print("공개키 생성")
p = int(input("첫번째 소수를 입력 : "))
q = int(input("두번째 소수를 입력 : "))
n = p*q
a = 2
pin =  (p-1)*(q-1)
data = []
so = []
while a < pin:
  for i in range(1, pin + 1):
    if (pin % i == 0) & (a % i == 0):
      data.append(i)
      gcd = i

  if gcd == 1:
    so.append(a)
    a = a +1
  else:
    a = a +13
k = len(so)
i = randrange(k) 
e = so[i]
d = 1
while (e*d)%pin != 1:
  d = d + 1
if e == d:
  d = d + 1
  while (e*d)%pin != 1:
    d = d + 1
print("공개키: (%d, %d)"% (int(n), int(e)))
print("개인키: (%d, %d)"% (int(n), int(d)))
print("공개키 (n, e)")
print("개인키 (n, d)")
print("n의 값 ", n)
print("e의 값 ", e)
print("d의 값 ", d)
code = []
original = []
num = 0
GH = 0
while num <= length :
  M = result[num]
  C = (M**e) % n
  original.append(C)
  GH = a1 + (C-1)*d2
  code.append(GH)
  num = num + 1
print("RSA 암호화 결과")
print(original)
print("암호문에 등차수열 적용")
print(code)

암호화할 문자 입력[가급적 영어로]:
Enter : I want to go to UNIST
아스키코드로 변환
[73, 32, 119, 97, 110, 116, 32, 116, 111, 32, 103, 111, 32, 116, 111, 32, 85, 78, 73, 83, 84]
공개키 생성
첫번째 소수를 입력 : 133
두번째 소수를 입력 : 71
공개키: (9443, 5611)
개인키: (9443, 331)
공개키 (n, e)
개인키 (n, d)
n의 값  9443
e의 값  5611
d의 값  331
RSA 암호화 결과
[8935, 5373, 2373, 8287, 3715, 4848, 5373, 4848, 8042, 5373, 1965, 8042, 5373, 4848, 8042, 5373, 9031, 9332, 8935, 881, 1680]
암호문에 등차수열 적용
[6084083, 3658361, 1615361, 5642795, 2529263, 3300836, 3658361, 3300836, 5475950, 3658361, 1337513, 5475950, 3658361, 3300836, 5475950, 3658361, 6149459, 6354440, 6084083, 599309, 1143428]


5. Client: 수신받은 암호문을 등차수열의 역연산을 거치고 개인키를 통해 해독한다

In [ ]:
#Client: 수신받은 자료를 해독한다.
def free_print(a):
  sen1 = ""
  for i in a:
    sen1 = sen1 + i
  return sen1
print("개인키 입력과정")
n = int(input("n값을 입력 : "))
d = int(input("d값을 입력 : "))
decode = []
kal = 0
C = 0
while kal <= length:
  last = code[kal]
  last1 = (last- a1)
  last2 = last1/d2
  C = int(last2 + 1)  
  M = (C**d) % n
  decode.append(chr(M))
  kal = kal + 1
print("최종 해독결과")
print(free_print(decode))

개인키 입력과정
n값을 입력 : 9443
d값을 입력 : 331
최종 해독결과
I want to go to UNIST


5. 도청자: 초항과 등차를 알 수 없으므로 해독 불가 - 기존 RSA 해독 방식만 적용 

In [ ]:
#도청자: 공개키만을 이용하여 정보해독 
def free_print(a):
  sen1 = ""
  for i in a:
    sen1 = sen1 + i
  return sen1
K = []
def getPrimaryNum_Eratos(n): 
  nums = [True] * (n + 1) 
  for i in range(2, len(nums) // 2 + 1): 
    if nums[i] == True: 
      for j in range(i+i, n, i): 
        nums[j] = False 
  return [i for i in range(2, n) if nums[i] == True]
print("n 값을 입력")
n = int(input()) 
print("e값을 입력")
e = int(input())
prime_nums = getPrimaryNum_Eratos(n + 1) 
# prime_nums_reverse = reversed(prime_nums) 
# print(prime_nums) 
if n in prime_nums: 
  print(n) 
else: 
  answers = [] 
  for prime_num in prime_nums: 
    if n % prime_num == 0:
       while(n % prime_num == 0): 
         answers.append(prime_num) 
         n = n // prime_num 
  for num in answers: K.append(num)
print("n 값을 입력하세요")
n = int(input()) 
print(K)
pin = (K[0]-1)*(K[1]-1)
print("Φ(n)의 값", pin)
d = 2 
while (e*d)%pin != 1:
  d = d + 1
print("d의 값 ", d)
decode = []
kal = 0
while kal <= length:
  C = code[kal]
  M = (C**d) % n
  decode.append(chr(M))
  kal = kal + 1
print(free_print(decode))

n 값을 입력
9443
e값을 입력
5611
n 값을 입력하세요
9443
[7, 19, 71]
Φ(n)의 값 108
d의 값  43
᧿ᨖᢷ≣ȏᢪᨖᢪᵩᨖ⁷ᵩᨖᢪᵩᨖፏ₮᧿̩ξ


////////////////////////////////////////////아래 코드는 위의 코드를 세분화 한것(무시해주세요...)/////////////////////////////////////////////////////

In [ ]:
#외부자: 공개키만을 이용하야 정보해독 
def free_print(a):
  sen1 = ""
  for i in a:
    sen1 = sen1 + i
  return sen1
K = []
def getPrimaryNum_Eratos(n): 
  nums = [True] * (n + 1) 
  for i in range(2, len(nums) // 2 + 1): 
    if nums[i] == True: 
      for j in range(i+i, n, i): 
        nums[j] = False 
  return [i for i in range(2, n) if nums[i] == True]
print("n 값을 입력하세요")
n = int(input()) 
print("e값을 입력해주세요")
e = int(input())
prime_nums = getPrimaryNum_Eratos(n + 1) 
# prime_nums_reverse = reversed(prime_nums) 
# print(prime_nums) 
if n in prime_nums: 
  print(n) 
else: 
  answers = [] 
  for prime_num in prime_nums: 
    if n % prime_num == 0:
       while(n % prime_num == 0): 
         answers.append(prime_num) 
         n = n // prime_num 
  for num in answers: K.append(num)
print("n 값을 입력하세요")
n = int(input()) 
print(K)
pin = (K[0]-1)*(K[1]-1)
print("Φ(n)의 값", pin)
d = 2 
while (e*d)%pin != 1:
  d = d + 1
print("d의 값 ", d)
decode = []
kal = 0
while kal <= length:
  C = code[kal]
  M = (C**d) % n
  decode.append(chr(M))
  kal = kal + 1
print(free_print(decode))

n 값을 입력하세요
1219
e값을 입력해주세요
85
n 값을 입력하세요
1219
[23, 53]
Φ(n)의 값 1144
d의 값  821
aˎɰrÑÑ΋ˀ΋Ñ΋ɫʈaÞв


In [ ]:
#Server 문자를 아스키코드로 변환
s = input("Enter : ")
result = []
for i in s:
  result.append(ord(i))
print(result)
length = len(result) - 1
#공개키, 개인키 만들기
from random import *
print("공개키 생성")
p = int(input("첫번째 소수를 입력하세요 : "))
q = int(input("두번째 소수를 입력하세요 : "))
n = p*q
a = 2
pin =  (p-1)*(q-1)
data = []
so = []
while a < pin:
  for i in range(1, pin + 1):
    if (pin % i == 0) & (a % i == 0):
      data.append(i)
      gcd = i

  if gcd == 1:
    so.append(a)
    a = a +1
  else:
    a = a +13
k = len(so)
i = randrange(k) 
e = so[i]
d = 1
while (e*d)%pin != 1:
  d = d + 1
if e == d:
  d = d + 1
  while (e*d)%pin != 1:
    d = d + 1
print("공개키 (n, e)")
print("개인키 (n, d)")
print("n의 값 ", n)
print("e의 값 ", e)
print("d의 값 ", d)
print("Φ(n)의 값", pin)
code = []
original = []
num = 0
while num <= length :
  M = result[num]
  C = (M**e) % n
  code.append(C)
  num = num + 1
print(code)

Enter : this is my password
[116, 104, 105, 115, 32, 105, 115, 32, 109, 121, 32, 112, 97, 115, 115, 119, 111, 114, 100]
공개키 생성
첫번째 소수를 입력하세요 : 107
두번째 소수를 입력하세요 : 101
공개키 (n, e)
개인키 (n, d)
n의 값  10807
e의 값  3041
d의 값  5361
Φ(n)의 값 10600
[5364, 8442, 6396, 8902, 5890, 6396, 8902, 5890, 9547, 3346, 5890, 4782, 2290, 8902, 8902, 7727, 8494, 8851, 8483]


In [ ]:
def free_print(a):
  sen1 = ""
  for i in a:
    sen1 = sen1 + i
  return sen1
#암호 복호화
n = int(input("n값을 입력하세요 : "))
d = int(input("d값을 입력하세요 : "))
decode = []
kal = 0
C = 0
print(code)
while kal <= length:
  C = code[kal]
  M = (C**d) % n
  decode.append(chr(M))
  kal = kal + 1
print(free_print(decode))

n값을 입력하세요 : 10807
d값을 입력하세요 : 5361
[5364, 8442, 6396, 8902, 5890, 6396, 8902, 5890, 9547, 3346, 5890, 4782, 2290, 8902, 8902, 7727, 8494, 8851, 8483]
this is my password


In [ ]:
#공개키, 개인키 만들기
from random import *
print("공개키 생성")
p = int(input("첫번째 소수를 입력하세요 : "))
q = int(input("두번째 소수를 입력하세요 : "))
n = p*q
a = 2
pin =  (p-1)*(q-1)
data = []
so = []
while a < pin:
  for i in range(1, pin + 1):
    if (pin % i == 0) & (a % i == 0):
      data.append(i)
      gcd = i

  if gcd == 1:
    so.append(a)
    a = a +1
  else:
    a = a +1
print(so)
k = len(so)
i = randrange(k) 
e = so[i]
d = 1
while (e*d)%pin != 1:
  d = d + 1
if e == d:
  d = d + 1
  while (e*d)%pin != 1:
    d = d + 1
print("공개키 (n, e)")
print("개인키 (n, d)")
print("n의 값 ", n)
print("e의 값 ", e)
print("d의 값 ", d)
print("Φ(n)의 값", pin)

공개키 생성
추천 소수 목록
2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71,
73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 
167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257,
263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359,
367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461,
463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577
587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677
683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809,
811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919,
929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031,
1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117,
1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 

In [ ]:
#정보 암호화 
print("암호화 수 설정시 유의사항")
print("1. 암호화핳 수는 n 보다 작야야합니다")
print("2. 암호화할 수가 n-1일 시에는 암호화할 수와 암호와 일치하는 경우가 발생합니다")
print("n 값", n) 
M = int(input("암호화 할 수를 입력하세요 : "))
C = (M**e) % n
print("암호: ", C)

암호화 수 설정시 유의사항
1. 암호화핳 수는 n 보다 작야야합니다
2. 암호화할 수가 n-1일 시에는 암호화할 수와 암호와 일치하는 경우가 발생합니다
n 값 6497
암호화 할 수를 입력하세요 : 6000
암호:  902


In [ ]:
# 암호 복호화
C = int(input("암호를 입력하세요 : "))
M = (C**d) % n
print("해독 결과: ", M)

암호를 입력하세요 : 902
해독 결과:  6000


In [ ]:
#정보 암호화 (분리)
M = int(input("암호화 할 수를 입력하세요 : "))
e = int(input("e값을 입력하세요 : "))
n = int(input("n값을 입력하세요 : "))
C = (M**e) % n
print("암호: ", C)

암호화 할 수를 입력하세요 : 6000
e값을 입력하세요 : 1835
n값을 입력하세요 : 6497
암호:  902
